<a href="https://www.kaggle.com/code/mikedelong/world-population-2023-data-import?scriptVersionId=139221648" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/world-population-by-country-2023/WorldPopulation2023.csv
/kaggle/input/country-mapping-iso-continent-region/continents2.csv


In [2]:
df = pd.read_csv('/kaggle/input/world-population-by-country-2023/WorldPopulation2023.csv')
df.head()

,Rank,Country,Population2023,YearlyChange,NetChange,Density(P/Km²),Land Area(Km²),Migrants(net),Fert.Rate,MedianAge,UrbanPop%,WorldShare
0,36,Afghanistan,42239854,2.70 %,1111083,65,652860,-65846,4.4,17.0,26 %,0.53 %
1,138,Albania,2832439,-0.35 %,-9882,103,27400,-8000,1.4,38.0,67 %,0.04 %
2,34,Algeria,45606480,1.57 %,703255,19,2381740,-9999,2.8,28.0,75 %,0.57 %
3,212,American Samoa,43914,-0.81 %,-359,220,200,-790,2.2,29.0,N.A.,0.00 %
4,202,Andorra,80088,0.33 %,264,170,470,200,1.1,43.0,85 %,0.00 %


In [3]:
df.tail()

,Rank,Country,Population2023,YearlyChange,NetChange,Density(P/Km²),Land Area(Km²),Migrants(net),Fert.Rate,MedianAge,UrbanPop%,WorldShare
229,225,Wallis & Futuna,11502,-0.60 %,-70,82,140,-119,1.9,37.0,0 %,0.00 %
230,172,Western Sahara,587259,1.96 %,11273,2,266000,5600,2.2,32.0,95 %,0.01 %
231,44,Yemen,34449825,2.24 %,753211,65,527970,-29914,3.6,19.0,37 %,0.43 %
232,63,Zambia,20569737,2.76 %,552062,28,743390,-5000,4.2,17.0,46 %,0.26 %
233,74,Zimbabwe,16665409,2.11 %,344872,43,386850,-9999,3.4,18.0,37 %,0.21 %


In [4]:
df.shape

(234, 12)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Rank            234 non-null    int64  
 1   Country         234 non-null    object 
 2   Population2023  234 non-null    int64  
 3   YearlyChange    234 non-null    object 
 4   NetChange       234 non-null    int64  
 5   Density(P/Km²)  234 non-null    int64  
 6   Land Area(Km²)  234 non-null    int64  
 7   Migrants(net)   234 non-null    int64  
 8   Fert.Rate       233 non-null    float64
 9   MedianAge       233 non-null    float64
 10  UrbanPop%       234 non-null    object 
 11  WorldShare      234 non-null    object 
dtypes: float64(2), int64(6), object(4)
memory usage: 22.1+ KB


In [6]:
df.describe()

,Rank,Population2023,NetChange,Density(P/Km²),Land Area(Km²),Migrants(net),Fert.Rate,MedianAge
count,234.000000,2.340000e+02,2.340000e+02,234.000000,2.340000e+02,2.340000e+02,233.000000,233.000000
mean,117.500000,3.437565e+07,3.000230e+05,477.414530,5.559568e+05,1.301282e+01,2.414163,31.309013
std,67.694165,1.373861e+08,1.001815e+06,2320.693692,1.691024e+06,1.698334e+05,1.155913,9.628386
min,1.000000,5.180000e+02,-2.957105e+06,0.000000,0.000000e+00,-9.104750e+05,0.800000,15.000000
25%,59.250000,4.696482e+05,2.360000e+02,38.250000,2.650000e+03,-9.776750e+03,1.600000,22.000000
50%,117.500000,5.643895e+06,2.860150e+04,96.500000,7.972000e+04,-5.000000e+02,2.000000,32.000000
75%,175.750000,2.324537e+07,2.236855e+05,242.000000,4.070800e+05,4.750000e+02,3.000000,40.000000
max,234.000000,1.428628e+09,1.145449e+07,24360.000000,1.637687e+07,1.784718e+06,6.700000,54.000000


Let's make some maps; first we need to get data that will let plotly place things on a map.

In [7]:
iso_df = pd.read_csv(filepath_or_buffer='/kaggle/input/country-mapping-iso-continent-region/continents2.csv').drop(columns=['alpha-2', 'intermediate-region', 'intermediate-region-code'])
iso_df.head()

,name,alpha-3,country-code,iso_3166-2,region,sub-region,region-code,sub-region-code
0,Afghanistan,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0
1,Åland Islands,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,150.0,154.0
2,Albania,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,150.0,39.0
3,Algeria,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,2.0,15.0
4,American Samoa,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,9.0,61.0


In [8]:
from plotly.express import choropleth
choropleth(data_frame=df[['Country', 'Fert.Rate']].merge(right=iso_df[['name', 'alpha-3',]], left_on='Country', right_on='name').drop(columns=['name']), color='Fert.Rate', locations='alpha-3', ).show()

In [9]:
choropleth(data_frame=df[['Country', 'MedianAge']].merge(right=iso_df[['name', 'alpha-3',]], left_on='Country', right_on='name').drop(columns=['name']), color='MedianAge', locations='alpha-3', ).show()